In [1]:
# # Standard imports


# # Custom imports
# from src.spotify_client import *

In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import re
import json
from datetime import datetime, time, date

import datetime
import time

from requests.exceptions import SSLError
from requests.exceptions import ConnectionError

# #Client identification for API requests
client_id = 'f78a4f4cfe9c40ea8fe346b0576e98ea'
client_secret = 'c26db2d4c1fb42d79dc99945b2360ab4'
def temp():
    print('it worked')

# Rough client for simplifying API calls
class SpotifyAPI(object):
    # Access toekn info that allows us to make requests
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    
    # Client information
    client_id = None
    client_secret = None
    
    # URL used for authentication and receiving an access token
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
        
    def get_client_credentials(self):
        '''
        returns a base 64 encoded string containing the client ID and Secret
        
        Converting to base 64 is necessary for the API requests
        '''
        client_id = self.client_id
        client_secret = self.client_secret

        if client_id == None or client_secret == None:
            raise Exception("Must set client_id and client_secret")
        
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_header(self):
        '''
        Creates the header for the authorization token request
        '''
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization' : f'Basic {client_creds_b64}' # Basic <base64 encoded client_id:client_secret>
        }
    
    def get_token_data(self):
        ''' 
        Creates the grant type for our application
        
        This changes what kind of data we can request from the API and what our rate limit
        for requests is.
        
        We use 'client_credentials' which doesn't allow us to access user information, but provides
        a higher rate limit for requests.
        '''
        return {
            'grant_type': 'client_credentials'
        }
    
    def auth(self):
        '''
        Follows the authorization flow for client_credentials to aquire an access token
        
        **CAUTION** The authorization token supposedly expires every hour even though
        the Spotify API documentation says otherwise. Out of caution we have implemented
        a feature that re-authenticates if an hour has passed since the last authentication
        '''
        
        # Information associated with requesting for an access token
        token_data = self.get_token_data()
        token_headers = self.get_token_header()
        token_url = self.token_url

        
        r = requests.post(token_url, data = token_data, headers = token_headers)
        
        # Raises an exception if there is some error with the authentication request
        if r.status_code not in range (200, 299):
            print("ERROR: Authentication failed with error code {}".format(r.status_code))
            return False
        
        # Convert response to a dictionary to parse
        token_response_data = r.json()
        
        # Save access token
        access_token = token_response_data['access_token']
        self.access_token = access_token

        # Save expiration time for access token
        now = datetime.datetime.now()
        expires_in = token_response_data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        
        return True
        
    def search_track(self, item):
        access_token = self.access_token
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': item, 'type':'track'})

        lookup_url = f'{endpoint}?{data}'
        # print(lookup_url)

        r = requests.get(lookup_url, headers=headers)
        # print(r.status_code)
        return r.json()
    
    def searchAlbum(self, item):
        access_token = self.access_token
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': item, 'type':'album'})

        lookup_url = f'{endpoint}?{data}'
        print(lookup_url)

        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        return r.json()
    
    def searchArtist(self, item):
        access_token = self.access_token
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': item, 'type':'artist'})

        lookup_url = f'{endpoint}?{data}'
        print(lookup_url)

        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        return r.json()

# Initialize the Client Object

In [2]:
client = SpotifyAPI(client_id, client_secret)
client.auth()

True

# Test the client by searching for individual things

In [3]:
r = client.searchArtist('the eden project')
r['artists']['items'][0]['genres']

https://api.spotify.com/v1/search?q=the+eden+project&type=artist
200


['electropop', 'indie poptimism', 'modern rock', 'pop']

# Combine Raw History With Spotify Information

### Variables

In [ ]:
data_path = 'data/data_2019'

### Load all files that contain streaming history information

In [ ]:
# Set Up Streaming History
streamingHistory = []

for f in os.listdir('data/data_2019'):
    if f.startswith('StreamingHistory'):
        streamingHistory.append(pd.read_json(os.path.join('data/data_2019', f)))
    
        
rawHist = pd.concat(streamingHistory).reset_index(drop=True)

### Aquire genre information by requesting metadata for each unique artist

Spotify labels artists with a variety of genres. This information can be aquired by requesting artist information from the API and parsing the json the API returns.

**ISSUES**

1. Sometimes an API request fails because of a bad request or communication issue. This shows up as an SSL error and the provided code tries to account for that. If a request fails because of an SSL error the program retrys the request until it is successful.


2. Sometimes the artist search returns nothing. To prevent the program from erroring we just check to see if the request returned at least one thing. If it did not return anything then we append NAN to our array of genres.

MAYBE CHANGE: It might make more sense to return an empty array instead of NAN but that can be changed at a later date.


3. I SWEAR there used to be an issue with some 'artists' having no genre information. This usually happened with very small artists, artists with strange characters in their name, and (all?) podcasts. I don't entirely remember how I corrected this issue because I don't see anything obvious in my code but if we find errors down the line then this may be a potential cause.


In [ ]:
artists = rawHist['artistName'].unique()
genreList = []
skippedList = []
painCount = 0
painOffset = 0
i = 0
while i < len(artists):
    try:
        print(i - painOffset)
        r = client.searchArtist(artists[i - painOffset])

        # Make sure the query returns something
        if len(r['artists']['items']) != 0:
            print('APPENDED GENRE')
            genreList.append(r['artists']['items'][0]['genres'])
        else:
            print('APPENDED NAN')
            genreList.append(np.nan)
            
        i += 1

        
    except SSLError:
        painCount += 1
        print("HELP HELP HELP HELP")
        
            
print(painCount)

In [ ]:
genreKey = dict(zip(artists, genreList))

In [ ]:
genreKey['Zedd']

In [ ]:
dfGenres = pd.DataFrame({'artistName':artists, 'genres':genreList})

In [ ]:
dfGenres.to_csv('dfGenres.csv', index=False)

In [ ]:
rawHist.to_csv('totalHist.csv', index=False)

In [ ]:
rawHist

In [ ]:
def appendGenres(row):
    artist = row['artistName']
    return genreKey[artist]
    

In [ ]:
genreCol = rawHist.apply(appendGenres, axis=1)
hist = rawHist.assign(genre = genreCol)


In [ ]:
hist

In [ ]:
enc = OneHotEncoder()

In [ ]:
hist = hist.dropna()
hist.isna().mean()

In [ ]:
temp = []
for i in genreKey.values():
    if type(i) == list:
        for j in i:
            temp.append(j)

In [ ]:
genres = pd.Series(temp).unique()

In [ ]:
datetime.date.today()